<a href="https://colab.research.google.com/github/huyennk98/Interactive-Dashboards-and-Data-Apps-with-Plotly-and-Dash/blob/master/THP_0104_8PM_return_Station.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ortools
import math  
import numpy as np   
import random as rd      
import matplotlib.pyplot as plt  
import numba  
from numba import njit  
import time  
from matplotlib.pyplot import figure

import ortools   
from ortools.constraint_solver import routing_enums_pb2     
from ortools.constraint_solver import pywrapcp   
from numba.typed import List   
from collections import defaultdict   
from matplotlib.pyplot import figure
import copy

import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 KB 10.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.12.0 requires protobuf<4,>=3.13, but you have protobuf 4.22.1 which is incompatible.
Mounted at /content/drive


In [2]:
%autosave 60
import json
from dataclasses import dataclass
from typing import List
# from datetime import datetime
from datetime import datetime as dt
import math

Autosaving every 60 seconds


In [3]:
# filename = 'fakePackmore2003.json'
# filename = 'pack_more_input_1903.json'
# filename = "fakePackmore2103.json"
# filename = 'PM2403.json'
# filename = 'packmore2503.json'
# filename = 'stm2803_1.json'
# filename = 'packmore2903.json'
# filename = "testxevedepot1.json"
# filename = "stm0104_S_1.json"
filename = "stm0104_S_2.json"

with open(filename) as f:
  data_origin_get_json = json.load(f)

In [4]:
class prepocessing:
    def __init__(self, json_input):
        self.json_input= json_input


# move tất cả các class lên đây cho anh

#Ultility

In [5]:

def compute_min_date_cus_depot(customers,depots):
    cus_depot = customers+depots

    number_request = len(cus_depot)

    cus_depot_working_time = np.zeros((number_request,2))

    list_time_working_veh = np.zeros((number_request,2), np.int_)
    temp= [ ]
    for index in range(number_request):

        each_veh = cus_depot[index]
        
        working_time_start = each_veh['workingTime']['start']

        working_time_end = each_veh['workingTime']['end']
    
        date_end, total_working_minutes_end = convert_to_date_obj(working_time_end)
        
        date_start, total_working_minutes_start = convert_to_date_obj(working_time_start)

        temp.append([date_start, total_working_minutes_start , date_end, total_working_minutes_end    ])

    min_date_cus = min([temp[i][0] for i in range(len(temp))]  )

    return min_date_cus

In [6]:
def dic_location_index( locations,locationCode_all):
    locationType = [loc["lTypes"] for loc in locations]
    index_by_location = []
    dic_location = {}
    dic_location_type = {}
    for index in range(len(locationCode_all)):
        
        index_by_location.append(locationCode_all[index])

        dic_location[locationCode_all[index]] = index

        dic_location_type[locationCode_all[index]] = locationType[index]
    
    return index_by_location, dic_location,dic_location_type

In [7]:
def convert_to_date_obj(s):
    yyyy= int(s[:4])
    mm= int(s[6:7])
    dd= int(s[8:10])
    hh= int(s[11:13])
    mininute= int(s[14:16])
    ss= int(s[17:19])
    x=  dt(yyyy, mm, dd,0).timestamp()
    return round(x), round(x+ 60*mininute + ss + hh*3600)  # 0: thời gian bắt đầu của ngày đó, còn 1 là thời gian thực sự.

#Depot


In [8]:
def combine_dictionary(depots, id_string):
    temp = { }
    for  req in depots:
        name_key_code = (req[id_string],)
        temp[name_key_code]= req
   
    
    return temp



In [9]:
def breaking_time_depot(depots,arr_key_depot,date_start):
    number_depot = len(depots)

    # print(depot_working_time)

    depot_break_time = [[]]*number_depot

    for index in range(number_depot):

        locationCode = arr_key_depot[index][0]

        for loc in depots:

            if loc["locationCode"] == locationCode:

                depot_break_time_interval =[]

                for break_time_interval in loc["breakTimes"]:

                    break_time_start = break_time_interval['start']

                    break_time_end = break_time_interval['end']
            
                    total_break_minutes_start = convert_to_date_obj(break_time_start)[1] -date_start
        
                    total_break_minutes_end = convert_to_date_obj(break_time_end)[1]- date_start 
            
                    depot_break_time_interval.append([total_break_minutes_start,total_break_minutes_end])
    
                    depot_break_time[index] = depot_break_time_interval

    return depot_break_time

In [10]:
def working_time_depot(depots,arr_key_depot,date_start):    
    depot_working_time = np.zeros((len(depots),2),  np.int_)

    for index in range(len(depots)):

        locationCode = arr_key_depot[index][0]

        for loc in depots:

            if loc["locationCode"] == locationCode:
    
                working_time_start = loc['workingTime']['start']

                working_time_end = loc['workingTime']['end']

                total_working_minutes_start = convert_to_date_obj(working_time_start)[1]- date_start 

                total_working_minutes_end = convert_to_date_obj(working_time_end)[1] -date_start
        
                depot_working_time[index] = np.array([total_working_minutes_start,total_working_minutes_end])

    return depot_working_time

#Customer

In [11]:
def convert(input):
    arr_key=  [ ]
    arr_key_CutomerCode = []
    output=  [ ]
    dict_to_list_id  = { }
    dict_to_list_id_by_cutomer_code = {}
    for key, val in input.items():
        arr_key.append(key)
        arr_key_CutomerCode.append(key[0])
        output.append(val)
        dict_to_list_id[key] =  len(arr_key)-1  # truyền vào key return index
        dict_to_list_id_by_cutomer_code[key[0]] = len(arr_key)-1

    return  output, arr_key,arr_key_CutomerCode,  dict_to_list_id ,dict_to_list_id_by_cutomer_code

In [12]:
def breaking_time_customer(dic_delivery_pickup,arr_key,customers,date_start):
    number_request = len(dic_delivery_pickup)

    # print(depot_working_time)

    cus_break_time = [[]]*number_request

    for index in range(number_request):

        locationCode = arr_key[index][0]

        for loc in customers:

            if loc["locationCode"] == locationCode:

                depot_break_time_interval =[]

                for break_time_interval in loc["breakTimes"]:

                    break_time_start = break_time_interval['start']

                    break_time_end = break_time_interval['end']
            
                    total_break_minutes_start = convert_to_date_obj(break_time_start)[1] -date_start
        
                    total_break_minutes_end = convert_to_date_obj(break_time_end)[1]- date_start 
            
                    depot_break_time_interval.append([total_break_minutes_start,total_break_minutes_end])
    
                    cus_break_time[index] = depot_break_time_interval

    return cus_break_time

In [13]:
def working_time_customer(dic_delivery_pickup,arr_key,customers,date_start):
    cus_working_time = np.zeros((len(dic_delivery_pickup),2),  np.int_)

    for index in range(len(dic_delivery_pickup)):

        locationCode = arr_key[index][0]

        for loc in customers:

            if loc["locationCode"] == locationCode:
    
                working_time_start = loc['workingTime']['start']

                working_time_end = loc['workingTime']['end']

                total_working_minutes_start = convert_to_date_obj(working_time_start)[1]- date_start 

                total_working_minutes_end = convert_to_date_obj(working_time_end)[1] -date_start
        
                cus_working_time[index] = np.array([total_working_minutes_start,total_working_minutes_end])

    return cus_working_time

In [14]:
def infor_customers(dic_delivery_pickup,arr_key,customers):
    infor_cus = []
    for index in range(len(dic_delivery_pickup)):

        locationCode = arr_key[index][0]

        for loc in customers:

            if loc["locationCode"] == locationCode:

                infor_cus.append(loc)

    return infor_cus


In [15]:
def combine_dictionary_arr(requests, id_string_arr):
    """

    """
    temp = { }

    for  req in requests:
        new_arr= tuple([req[x] for x in id_string_arr])
        temp[new_arr]= [ ]
    for  req in requests:
        new_arr= tuple([req[x] for x in id_string_arr])
        temp[new_arr].append( req)
    return temp

#Vehicle

In [16]:
def combine_dictionary_veh(vehicles, id_string):
    temp = { }
    for  req in vehicles:
        temp[req[id_string]]= [ ]
    for  req in vehicles:
        temp[req[id_string]].append( req)
    return temp


In [17]:
def working_time_vehicle(vehicles,date_start,dict_to_list_id_veh):
    number_of_vehicle = len(vehicles)
    list_time_working_veh = np.zeros((number_of_vehicle,2), np.int_)

    for index in range(number_of_vehicle):

        each_veh = vehicles[index]
        
        working_time_start = each_veh['workingTime']['start']

        working_time_end = each_veh['workingTime']['end']

        total_working_minutes_end = convert_to_date_obj(working_time_end)[1] -date_start
        
        total_working_minutes_start = convert_to_date_obj(working_time_start)[1]- date_start 
        print(dict_to_list_id_veh)
        print( each_veh['vehicleCode'])
        i = dict_to_list_id_veh[each_veh['vehicleCode']]

        list_time_working_veh[i] = np.array([total_working_minutes_start,total_working_minutes_end])

    return list_time_working_veh

In [18]:
# vehicle capacity/cbmd
def compute_list_cbm_veh_list_capacity_veh(vehicles):

    number_of_vehicle = len(vehicles)

    capacity_veh = vehicles[0]['capacity']
    cbm_veh = vehicles[0]['cbm']
    list_capacity_veh = np.zeros((number_of_vehicle,), np.int_)
    list_cbm_veh = np.zeros((number_of_vehicle,), np.int_)

    for index_v in range(number_of_vehicle):
        list_cbm_veh[index_v] = vehicles[index_v]['cbm']
        list_capacity_veh[index_v] = vehicles[index_v]['capacity']
    return list_cbm_veh, list_capacity_veh

In [19]:
def convert_vehicle(input):
    arr_key_veh=  [ ]
    
    output_veh=  [ ]
    
    dict_to_list_id_veh = {}
    for key, val in input.items():
        arr_key_veh.append(key)
 
        output_veh.append(val)

        dict_to_list_id_veh[key] =  len(arr_key_veh)-1  # truyền vào key return index
     
    return arr_key_veh,output_veh,dict_to_list_id_veh



In [20]:
def compute_min_date_veh(vehicles):
    number_of_vehicle = len(vehicles)
    list_time_working_veh = np.zeros((number_of_vehicle,2), np.int_)
    temp= [ ]
    for index in range(number_of_vehicle):

        each_veh = vehicles[index]
        
        working_time_start = each_veh['workingTime']['start']

        working_time_end = each_veh['workingTime']['end']
    
        date_end, total_working_minutes_end = convert_to_date_obj(working_time_end)
        
        date_start, total_working_minutes_start = convert_to_date_obj(working_time_start)

        temp.append([date_start, total_working_minutes_start , date_end, total_working_minutes_end    ])

    min_date_veh = min([temp[i][0] for i in range(len(temp))]  )
    return min_date_veh

In [21]:
# workingtime and breaktime of depot
def break_time_vehicle(vehicles,date_start,dict_to_list_id_veh):
    number_request = len(vehicles)

    # print(depot_working_time)

    vehicle_break_time = [[]]*number_request

    for index in range(number_request):

        each_depot = vehicles[index]
        
        depot_break_time_interval =[]

        for break_time_interval in each_depot["breakTimes"]:

            break_time_start = break_time_interval['start']

            break_time_end = break_time_interval['end']
            
            total_break_minutes_start = convert_to_date_obj(break_time_start)[1] -date_start
        
            total_break_minutes_end = convert_to_date_obj(break_time_end)[1]- date_start 
            
            depot_break_time_interval.append([total_break_minutes_start,total_break_minutes_end])
            
            # print("depot_break_time_interval",depot_break_time_interval)

        i = dict_to_list_id_veh[each_depot['vehicleCode']]
    
        vehicle_break_time[i] = depot_break_time_interval
    return vehicle_break_time

#Main

In [22]:
@dataclass
class OriginVehicle:
    id: str
    

    vehicleCode: str
    startLocationCode: str
    startLocationType: str
    endLocationCode: str
    endLocationType: str

    working_time: list[np.ndarray]
    breaking_time: list
 
    cbm: int

    capacity: int

    infor_veh: list

    state: dict[str, List[np.ndarray]] # {"TW_list_free":[],"TW_used":[], "time_list_cus":[time_cus_by_vehicle_id]} 

@dataclass
class OriginCustomer:
    id: str
    index_origin: int
    CustomerCode: str
    locationCode: str
    working_time:  list[np.ndarray]
    break_time: List[np.ndarray]
    infor_cus: list
    infor_req: list
    # cbm_req: int
    # capacity_req: int
    state: dict[str, int] # {"trip": int, "vehicle", "time_start":time_start}

@dataclass
class OriginDepot:
    id: str
    index_origin:int
    depotCode: str
    locationCode: str
    working_time:  list[np.ndarray]
    break_time: List[np.ndarray]
    infor_depot : list
    state: dict[str, int] # {"trip": int, "vehicle", "time_start":time_start}


def processing_input(data_origin_get_json):
    customers = data_origin_get_json['customers']
    locations = data_origin_get_json['locations']
    requests = data_origin_get_json['requests']
    vehicles = data_origin_get_json['vehicles']
    depots = data_origin_get_json['depots']
    distances = data_origin_get_json['distances']
    matrixConfig = data_origin_get_json['matrixConfig']

    for i in range(len(requests)):

        for item in requests[i]['items']:
    
            item['orderCode']= requests[i]['orderCode']

    # algoParams = data_origin_get_json['algoParams']
    locationCode_all = [loc["locationCode"] for loc in locations]

    index_by_location, dic_location,dic_location_type = dic_location_index(locations,locationCode_all)
    num_location = len(locationCode_all)
    DM = np.zeros(  (num_location,num_location), np.int_)
    TM = np.zeros( (num_location,num_location), np.int_)
# code ngu cực!
    for i in range(num_location):

        locationCode_src = index_by_location[i]

        for j in range(num_location):

            locationCode_dest = index_by_location[j]

            for pair_loc in distances:

                if pair_loc['srcCode'] == locationCode_src and pair_loc['destCode'] == locationCode_dest:
                    
                    DM[i,j] = pair_loc["distance"]*1000

                
                    TM[i,j] = pair_loc['travelTime']




# Tính thời gian bắt đầu sớm nhất của customer và depot
    min_date_cus = compute_min_date_cus_depot(customers,depots)
    min_date_veh = compute_min_date_veh(vehicles)
    date_start = max(min_date_veh,min_date_cus)

    #Vehicle
    capacity_veh = vehicles[0]['capacity']
    cbm_veh = vehicles[0]['cbm']
    list_cbm_veh, list_capacity_veh = compute_list_cbm_veh_list_capacity_veh(vehicles)
    dictionary_by_vehicleCode =combine_dictionary_veh(vehicles, 'vehicleCode')
    for veh in vehicles:
        print(veh)
    print(dictionary_by_vehicleCode)
   

    arr_key_veh,output_veh,dict_to_list_id_veh = convert_vehicle(dictionary_by_vehicleCode)
    # print(dict_to_list_id_veh)
    list_time_working_veh = working_time_vehicle(vehicles,date_start,dict_to_list_id_veh)
    vehicle_break_time = break_time_vehicle(vehicles,date_start,dict_to_list_id_veh)

    
    vehicle_list = []
    number_of_vehicle = len(vehicles)
    start_LocationCode_value = [veh["startLocationCode"] for veh in vehicles]
    start_LocationCode_type =  [veh["startLocationType"] for veh in vehicles]
    end_LocationCode_value = [veh["endLocationCode"] for veh in vehicles]
    end_LocationCode_type = [veh["endLocationType"] for veh in vehicles]
    for index in range(number_of_vehicle):
    
        vehicle_list.append( OriginVehicle( arr_key_veh[index], arr_key_veh[index], start_LocationCode_value[index], \
                                           start_LocationCode_type[index], \
                                    end_LocationCode_value[index], end_LocationCode_type[index], list_time_working_veh[index],\
                                     vehicle_break_time[index], \
                                    list_cbm_veh[index], list_capacity_veh[index],dictionary_by_vehicleCode[arr_key_veh[index]],{}\
                                    )
                            )
    
    #Customer_request
    dic_delivery_pickup =combine_dictionary_arr(requests, ['deliveryLocationCode', "pickupLocationCode"])
    
    output, arr_key,arr_key_CutomerCode,  dict_to_list_id ,dict_to_list_id_by_cutomer_code = convert(dic_delivery_pickup)
    cus_working_time = working_time_customer(dic_delivery_pickup,arr_key,customers,date_start)
    cus_break_time = breaking_time_customer(dic_delivery_pickup,arr_key,customers,date_start)
    infor_cus = infor_customers(dic_delivery_pickup,arr_key,customers)

    customer_list = [ ]
    for index in range(len(dic_delivery_pickup)):
        customer_list.append(OriginCustomer(arr_key[index],dic_location[arr_key_CutomerCode[index]] ,dic_delivery_pickup[arr_key[index]][0]['customerCode'],\
                                            arr_key_CutomerCode[index],cus_working_time[index],\
                                            cus_break_time[index],\
                                            infor_cus[index],
                                            dic_delivery_pickup[arr_key[index]],
                                            {}

        ))

    # depot list
    dic_depot =combine_dictionary(depots, 'locationCode')
    output_depot, arr_key_depot,arr_key_DepotCode,  dict_to_list_id_depot ,dict_to_list_id_by_depot_code = convert(dic_depot)
    depot_working_time = working_time_depot(depots,arr_key_depot,date_start)
    depot_break_time = breaking_time_depot(depots,arr_key_depot,date_start)
    depot_list_origin = [ ]
    for index in range(len(dic_depot)):
        depot_list_origin.append(OriginDepot(arr_key_depot[index], dic_location[arr_key_depot[index][0]]  ,dic_depot[arr_key_depot[index]]['depotCode'],\
                                    arr_key_depot[index][0],depot_working_time[index],\
                                        depot_break_time[index],\
                                    dic_depot[arr_key_depot[index]],\
                                    {}

        ))
    
    return vehicle_list, customer_list, depot_list_origin, DM, TM,index_by_location, dic_location,dic_location_type,date_start, number_of_vehicle

vehicle_list, customer_list, depot_list_origin, DM, TM ,index_by_location, dic_location,dic_location_type, date_start, number_of_vehicle= processing_input(data_origin_get_json)
# vehicle_list

{'vehicleCode': '95H-00609_000', 'romoocCode': '61R-05059', 'cbm': 96343800, 'capacity': 40000000, 'quantity': 1, 'size': {'width': 1356, 'length': 245, 'height': 290}, 'loadTimePerCbm': 0, 'unloadTimePerCbm': 0, 'loadTimePerTon': 0, 'unloadTimePerTon': 0, 'startLocationCode': '4000', 'startLocationType': 'STATION', 'endLocationCode': '4000', 'endLocationType': 'STATION', 'workingTime': {'start': '2023-04-01 11:40:57', 'end': '2023-04-03 00:00:00'}, 'breakTimes': [], 'vType': {'typeOfVehicleByMultipleTrips': 'NC', 'typeOfVehicleToItemRule': 'TRAILER', 'typeOfVehicleByCostToDeploy': 'Đầu kéo', 'typeOfVehicleByPricePerKm': 'Đầu kéo', 'typeOfVehicleByLimits': 'IH', 'typeOfVehicleByLimitedHour': '40000000'}}
{'vehicleCode': '95H-00399_000', 'romoocCode': '61R-04839', 'cbm': 68300800, 'capacity': 40000000, 'quantity': 1, 'size': {'width': 1160, 'length': 230, 'height': 256}, 'loadTimePerCbm': 0, 'unloadTimePerCbm': 0, 'loadTimePerTon': 0, 'unloadTimePerTon': 0, 'startLocationCode': '8000000

In [23]:
endLocationCode_veh = []
for veh in vehicle_list:
    if veh.endLocationType == "STATION":
        endLocationCode_veh.append(veh.endLocationCode)
endLocationCode_veh

['4000', '8000000']

In [24]:
stationCode_list = []
for loactionCode, locationType in dic_location_type.items():
    if loactionCode in endLocationCode_veh and len(locationType) == 1 and 'STATION' in locationType:
        stationCode_list.append(loactionCode)
stationCode_list

['4000', '8000000']

In [25]:
depot_list = copy.deepcopy(depot_list_origin)
for station in stationCode_list:
    depot_list.append(OriginDepot( (station,),
                                  dic_location[station], station,station,np.array([0,999999999]),[],{},{}
                                  ))
depot_list

[OriginDepot(id=('4100',), index_origin=0, depotCode='4100', locationCode='4100', working_time=array([  1800, 257400]), break_time=[[43200, 46800], [129600, 133200]], infor_depot={'depotCode': '4100', 'locationCode': '4100', 'fixedLoadTime': 900, 'loadTimePerTon': 900, 'loadTimePerCbm': 36, 'workingTime': {'start': '2023-04-01 00:30:00', 'end': '2023-04-03 23:30:00'}, 'breakTimes': [{'start': '2023-04-01 12:00:00', 'end': '2023-04-01 13:00:00'}, {'start': '2023-04-02 12:00:00', 'end': '2023-04-02 13:00:00'}], 'dType': {'typeOfDepotByCapacity': None, 'typeOfLocationByLimitedHour': '4100'}}, state={}),
 OriginDepot(id=('4000',), index_origin=4, depotCode='4000', locationCode='4000', working_time=array([        0, 999999999]), break_time=[], infor_depot={}, state={}),
 OriginDepot(id=('8000000',), index_origin=5, depotCode='8000000', locationCode='8000000', working_time=array([        0, 999999999]), break_time=[], infor_depot={}, state={})]

# Cluster by depot

In [26]:
def find_depot_for_vehicle(depot_list_origin, start_veh_index,end_veh_index):
    
    total_distance = 9999999999999999
    depot_found = depot_list_origin[0]
    
    for depot in depot_list_origin:
        
        index_depot = depot.index_origin 
        
        distance_startVeh_depot_endVeh = TM[start_veh_index,index_depot] + TM[index_depot,end_veh_index]*0.2 
        
        if  distance_startVeh_depot_endVeh < total_distance:
            
            total_distance = distance_startVeh_depot_endVeh

            depot_found= depot.locationCode

    return depot_found, total_distance


In [27]:

def split_vehicle_list(vehicle_list,depot_list_origin):

    dic_veh_by_depot ={}
    
    for veh in vehicle_list:

        start_veh_code = veh.startLocationCode
        end_veh_code = veh.startLocationCode
        
        start_veh_index = dic_location[start_veh_code]
        end_veh_index = dic_location[end_veh_code]

        depot_found, total_distance = find_depot_for_vehicle(depot_list_origin, start_veh_index,end_veh_index)
    
        dic_veh_by_depot[veh.id] = depot_found

    return dic_veh_by_depot
dic_veh_by_depot = split_vehicle_list(vehicle_list,depot_list_origin)
dic_veh_by_depot

{'95H-00609_000': '4100', '95H-00399_000': '4100'}

In [28]:
def split_vehicle_by_depot(vehicle_list,depot_list_origin):

    dic_depot_by_veh = {}

    dic_veh_by_depot = split_vehicle_list(vehicle_list,depot_list_origin)

    for veh, depot in  dic_veh_by_depot.items():
        
        dic_depot_by_veh[depot] = []
    
    for veh, depot in  dic_veh_by_depot.items():

        for vehicle in vehicle_list:
            if vehicle.id ==   veh:
                dic_depot_by_veh[depot].append(vehicle)

        
        
    return dic_depot_by_veh
dic_depot_by_veh = split_vehicle_by_depot(vehicle_list,depot_list_origin)

In [29]:
def split_customer_by_depot(depot_startLocationCode, customer_list):
    customer_list_by_depot = []
    for cus in customer_list:
        id_request = cus.id
        if id_request[1] == depot_startLocationCode:
            customer_list_by_depot.append(cus)
    return customer_list_by_depot

In [30]:
def split_all_by_depot(depot_list_origin, customer_list, vehicle_list):
    
    all_VDC_by_depot = []
    
    
    for depot in depot_list_origin:
        
        VDC_by_depot = {}
        
        id_depot = depot.locationCode

        VDC_by_depot["vehicle_list_by_depot"] = split_vehicle_by_depot(vehicle_list,depot_list_origin)[id_depot]

        VDC_by_depot["customer_list_by_depot"] = split_customer_by_depot(id_depot, customer_list)

        VDC_by_depot["depot_list_new"] = depot

        all_VDC_by_depot.append(VDC_by_depot)
    
    return all_VDC_by_depot

VDC_new =  split_all_by_depot(depot_list_origin, customer_list, vehicle_list)


# Split request

In [31]:
def vehicle_after_split(vehicle_list_by_depot):
    dic_vehicle_after_split={}
    for veh in vehicle_list_by_depot:
        dic_vehicle_after_split[veh.vehicleCode] = veh.infor_veh
    return dic_vehicle_after_split

In [32]:
def working_time_vehicle(vehicle_list_by_depot,depot_list_new ):

    list_time_working_veh = np.zeros((len(vehicle_list_by_depot),2), np.int_)

    for index_veh in range(len(vehicle_list_by_depot)):
        veh = vehicle_list_by_depot[index_veh]
        startLocationCode = veh.startLocationCode

        locationCode_depot_start = depot_list_new.locationCode

        index_startLocationCode = dic_location[startLocationCode]

        index_locationCode_depot_start = dic_location[locationCode_depot_start]


        working_time_origin = veh.working_time

        working_time_start = working_time_origin[0] + TM[index_startLocationCode,index_locationCode_depot_start]
        working_time_end = working_time_origin[1]

    
        list_time_working_veh[index_veh] = np.array([working_time_start,working_time_end])

    return list_time_working_veh

In [33]:
def split_request_all(dic_delivery_pickup,max_cbm, max_weight):


    def filter_item(dic_delivery_pickup,max_cbm, max_weight ):
    
        new_request = copy.deepcopy(dic_delivery_pickup)

        drop_items = []



        # lọc bỏ những items có weight/cbm > weight/capacity of vehicle
        for code_cus_depot, val in dic_delivery_pickup.items():

            item_of_each_code_cus_depot=[]

        

            for index_req in range(len(dic_delivery_pickup[code_cus_depot])):

                req = dic_delivery_pickup[code_cus_depot][index_req]



                for index_item in range(len(req['items'])):

                    each_item = req['items'][index_item]      

                    each_item['cbm'] = each_item['cbm'] / each_item['quantity']

                    each_item['weight'] = each_item['weight'] / each_item['quantity']

                    if each_item['weight'] > max_weight or each_item['cbm'] > max_cbm:

                        # print(each_item)

                        new_request[code_cus_depot][index_req]['items'].remove(each_item)

                        drop_items.append([code_cus_depot,req,each_item])
                    # print("list_items",new_request[code_cus_depot][index_req]['items'])
        
        return new_request, drop_items
    new_request, drop_items = filter_item(dic_delivery_pickup,max_cbm, max_weight )

    def divide_items(items, max_cbm, max_weight):

        for item in items:
            item['cbm'] = round(item['cbm'] / item['quantity'])
            item['weight'] = round(item['weight'] / item['quantity'])

    
        
        # Group items by size
        grouped_items = {}
        for item in items:
            size = (item['cbm'], item['weight'])
            if size not in grouped_items:
                grouped_items[size] = []
            grouped_items[size].append(item)
        
        packages = []
        current_package = {'items': [], 'cbm': 0, 'weight': 0}
        
        for size, items in grouped_items.items():
            # Sort the items by their quantity
            sorted_items = sorted(items, key=lambda x: x['quantity'], reverse=True)
            
            for item in sorted_items:
                # Calculate the number of packages needed for this item
                num_packages = math.ceil(item['quantity'] )
                
                for i in range(num_packages):
                    if (current_package['cbm'] + item['cbm']) > max_cbm or (current_package['weight'] + item['weight']) > max_weight:
                        # Start a new package if adding the item would exceed the limits
                        packages.append(current_package)
                        current_package = {'items': [], 'cbm': 0, 'weight': 0}
                    
                    current_package['items'].append(item)
                    current_package['cbm'] += item['cbm']
                    current_package['weight'] += item['weight']
                
        
        # Add the last package to the list
        packages.append(current_package)
        
        return packages
    
    def split_request( new_request,  max_cbm, max_weight   ): # hope_to_split
        # max_cbm1=1000000000000
        # max_weight1=  100000000000
        dict_split_request = {}

        for code_cus_depot, val in new_request.items():

            item_of_each_code_cus_depot=[]

        
            for index_req in range(len(new_request[code_cus_depot])):

                req = new_request[code_cus_depot][index_req]

                item_of_each_code_cus_depot.extend(req['items'])

            packages = divide_items(item_of_each_code_cus_depot, max_cbm, max_weight)

            for item_split_index in range(len(packages)):

                list_item = packages[item_split_index]

                name_key_code_index = code_cus_depot + (item_split_index,)
            
                dict_split_request[name_key_code_index] = list_item

        return dict_split_request

    dict_split_request = split_request( new_request,  max_cbm, max_weight   )

    return new_request,dict_split_request, drop_items

In [34]:
def start_end_locationCode(depot_list_new, vehicle_list_by_depot,dict_to_list_id_split):

    start_LocationCode_value = [depot_list_new.locationCode]*len(vehicle_list_by_depot)
    end_LocationCode_value = [veh.endLocationCode for veh in vehicle_list_by_depot]
    
    start_LocationCode=[]
    end_LocationCode = []

    for start in start_LocationCode_value:
        for key,val in dict_to_list_id_split.items():
            
            if key[0] == start:
                start_LocationCode.append(val)

    for end in end_LocationCode_value:
        for key,val in dict_to_list_id_split.items():
            if key[0] == end:
                end_LocationCode.append(val)
    
    return start_LocationCode,end_LocationCode

In [35]:
def DM_TM_each_depot(dict_to_list_id_split, DM, TM,arr_key_split):
    # đặt lại tên
    number_request = len(dict_to_list_id_split)
    DM_each_depot = np.zeros(  (number_request,number_request), np.int_)
    TM_each_depot = np.zeros( (number_request,number_request), np.int_)

    for i in range(number_request):
        locationCode_i = arr_key_split[i][0]
        index_i = dic_location[locationCode_i]

        for j in range(number_request):
            locationCode_j = arr_key_split[j][0]
            index_j = dic_location[locationCode_j]

            DM_each_depot[i,j] = DM[index_i,index_j]
            TM_each_depot[i,j] = TM[index_i,index_j]

    return DM_each_depot, TM_each_depot

In [36]:
def cbm_weight_each_request_for_each_depot(dict_split_request_each_depot,depot_list):

    weight_request = [req['weight'] for key_code, req in dict_split_request_each_depot.items()]
    weight_each_depot = []
    for index_depot in range(len(depot_list)):
        weight_each_depot.append(0)

    weight_each_request = weight_each_depot+ weight_request

    cbm_request = [req['cbm'] for key_code, req in dict_split_request_each_depot.items()]

    cbm_each_depot = []
    for index_depot in range(len(depot_list)):
        cbm_each_depot.append(0)
    cbm_each_request = cbm_each_depot + cbm_request
    
    return weight_each_request, cbm_each_request

In [37]:
def service_time_depot(depot_list_new,cbm_veh,capacity_veh):

    loadTimePerCbm = depot_list_new.infor_depot['loadTimePerCbm']
    loadTimePerTon = depot_list_new.infor_depot['loadTimePerTon']
    servie_time_depot = max(loadTimePerCbm*cbm_veh/1_000_000, loadTimePerTon*capacity_veh/1_000_000)
    print( "time load hàng tại kho", servie_time_depot)
    return servie_time_depot

In [38]:
def working_time_customer_depot_for_each_depot(dict_to_list_id_split,customer_list_by_depot,depot_list,arr_key_split):

    cus_depot = depot_list + customer_list_by_depot

    number_request = len(dict_to_list_id_split)

    cus_depot_working_time = np.zeros((number_request,2),  np.int_)

    cus_depot_break_time = [[]]*number_request

    for index in range(number_request):
        locationCode = arr_key_split[index][0]
        
        for loc in cus_depot:
            if loc.id[0] == locationCode:
                cus_depot_working_time[index] = loc.working_time
                
                cus_depot_break_time[index] = loc.break_time
    # cus_depot_working_time
    return cus_depot_working_time,cus_depot_break_time

In [39]:
def timeService_of_customer_depot(dict_to_list_id_split,arr_key_split,customer_list_by_depot,\
                                  depot_list,weight_each_request,cbm_each_request,servie_time_depot):
# compute unload time
    cus_depot = depot_list + customer_list_by_depot

    number_request = len(dict_to_list_id_split)

    number_depot = len(depot_list)

    total_service_time = np.zeros((number_request,), np.int_)
    fixed_time = np.zeros((number_request,), np.int_)

    for index in range(number_depot, number_request):

        locationCode = arr_key_split[index][0]

        for loc in cus_depot:

            if loc.id[0] == locationCode:

                each_cus = loc.infor_cus

                fixedUnloadTime = each_cus['fixedUnloadTime']

                unloadTimePerTon = each_cus['unloadTimePerTon']

                unloadTimePerCbm = each_cus['unloadTimePerCbm']


                time_unload_by_Ton = round(((weight_each_request[index]/1_000_000)*unloadTimePerTon))
             

                time_unload_by_cbm = round(((cbm_each_request[index]/1_000_000)*unloadTimePerCbm))
            

                total_service_time[index] = max(time_unload_by_Ton,time_unload_by_cbm)

                # total_service_time[index] =  max(time_unload_by_Ton,time_unload_by_cbm)

                fixed_time[index] = fixedUnloadTime

    for index in range(number_depot):

        locationCode = arr_key_split[index][0]

        for loc in cus_depot:

            if loc.id[0] == locationCode:

                each_cus_depot = loc.infor_depot

                if each_cus_depot == {}:
                    
                    fixedLoadTime =0

                    total_service_time[index] = 0

                    fixed_time[index] = 0
                    
                else:

                    fixedLoadTime = each_cus_depot['fixedLoadTime']

                    total_service_time[index] =  servie_time_depot

                    fixed_time[index] = fixedLoadTime

    return total_service_time, fixed_time

In [40]:
def f_matrix_service_time(dict_to_list_id_split,total_service_time,fixed_time,depot_list):

    number_request = len(dict_to_list_id_split)
    matrix_service_time =  np.zeros((number_request,number_request), np.int_)

    number_depot = len(depot_list)
    print(number_depot)
    for i in range(number_request):

        for j in range( number_request):

            if i == j:
                matrix_service_time[i,j] =0
                continue

            if i < number_depot and j < number_depot:
                # i là depot, j là depot
                matrix_service_time[i,j] = 0
                continue

            if i < number_depot and j >= number_depot:
                # i là depot, j là cus

                matrix_service_time[i,j] = total_service_time[i] + \
                total_service_time[j] +fixed_time[j]
                print(                  matrix_service_time[i,j] , total_service_time[i] ,
                total_service_time[j] , fixed_time[j] )
                continue
            
            if i >= number_depot and j < number_depot:
                # i là cus, j là depot

                matrix_service_time[i,j] = fixed_time[j]
                # tất cả tính theo thời gian rời đi. 
                continue
            # i là cus, j là cus
            if i >= number_depot and j >= number_depot:

                matrix_service_time[i,j] = total_service_time[j] +fixed_time[j]
                continue
    print( matrix_service_time)
    return matrix_service_time

In [41]:
def preprocessing(DM_each_depot,time_matrix_all,cus_depot_working_time,cus_depot_break_time, capacity_veh,\
                  cbm_veh,number_of_vehicle,list_time_working_veh,cbm_each_request,weight_each_request, start_LocationCode, end_LocationCode
                  ):

    " create input new with data after split request"
    
    input_after_split ={}


    input_after_split['distance_matrix'] = np.copy(DM_each_depot)

    input_after_split['time_matrix'] = np.copy(time_matrix_all)      

    input_after_split['time_windows'] = np.copy(cus_depot_working_time)

    input_after_split['break_time_cus_depot'] = copy.deepcopy(cus_depot_break_time)

    input_after_split['capacity_veh'] = capacity_veh

    input_after_split['cbm_veh'] = cbm_veh

    input_after_split['num_veh'] = number_of_vehicle

    input_after_split['time_windows_veh'] = np.copy(list_time_working_veh)


    input_after_split['cbm_new'] = np.array(cbm_each_request, np.int_)

    input_after_split['weight'] = np.array(weight_each_request, np.int_)

    input_after_split['starts'] =  copy.deepcopy(start_LocationCode)

    input_after_split['ends'] = copy.deepcopy(end_LocationCode)
    # input_after_split['convert_from_split_to_origin_node'] = copy.deepcopy(convert_from_split_to_origin_node)

    # input_after_split['loc'] = np.copy(location_get_input)

    # input_after_split["STARTPOINT"] = STARTPOINT

    # input_after_split["ENDPOINT"] = ENDPOINT

    return input_after_split

In [42]:
@dataclass
class VehicleAfterSplit:
    
    id: int
    vehicleCode: str

    startLocationCode: int

    endLocationCode:int

    
    tw_list:  List[np.ndarray]
    cbm: int
    capacity: int
    state: dict[str, List[np.ndarray]] # {"TW_list_free":[],"TW_used":[], "time_list_cus":[time_cus_by_vehicle_id]} 
    """
    TW_list_free: tw của xe còn rỗi
    TW_used: TW của xe đã sử dụng
    time_list_cus: time_cus_by_vehicle_id
    """
 
    def print(self):
        print("id:", self.id, self.vehicleCode, self.startLocationCode, self.endLocationCode, \
              "time windows: " ,self.tw_list, self.cbm ," capacity:", self.capacity, "state:", self.state)

@dataclass
class CustomerAfterSplit:
    id:int
    customerCode: str
    tw_list:  List[np.ndarray]
    weight: int
    cbm: int

    break_time_customer: List[np.ndarray]
    state: dict[str, int] # {"trip": int, "vehicle", "time_start":time_start}
    is_visited:int = -1
    # -1 if unvisited, and take vehicle_id if vehicle_id visits this point

    def print(self):
        if self.is_visited==-1:
            print("id:", self.id,"time windows: " , self.tw_list," cbm:", self.cbm,self.break_time_customer, "not visited")
        else:
            print("id:", self.id,"time windows: " , self.tw_list," cbm:", self.cbm,self.break_time_customer, "visited by vehicle", self.is_visited)


## tiền xử lý

In [43]:
# customer_list_by_depot = VDC_new[0]['customer_list_by_depot']
# vehicle_list_by_depot = VDC_new[0]['vehicle_list_by_depot']
# depot_list_new = VDC_new[0]['depot_list_new']

In [44]:

def function_split_request(customer_list_by_depot,vehicle_list_by_depot,depot_list_new):

    max_cbm = vehicle_list_by_depot[0].cbm
    print("max_cbm",type(max_cbm))

    max_weight = vehicle_list_by_depot[0].capacity
    print("max_w",type(max_weight))

    #Vehicle
    dic_vehicle_after_split = vehicle_after_split(vehicle_list_by_depot)
    arr_key_veh_split,output_veh_split,dict_to_list_id_veh_split = convert_vehicle(dic_vehicle_after_split)
    list_time_working_veh = working_time_vehicle(vehicle_list_by_depot,depot_list_new)

    #Customer request depot
    dic_delivery_pickup_depot={}
    for cus_req in customer_list_by_depot:
        # print(cus_req)
        dic_delivery_pickup_depot[cus_req.id] = cus_req.infor_req

    new_request_each_depot, dict_split_request_each_depot, drop_items_each_depot = \
            split_request_all(dic_delivery_pickup_depot,max_cbm, max_weight)

    print(dict_split_request_each_depot)

    dic_depot ={}
    for depot in depot_list:
        dic_depot[depot.id] = depot.infor_depot

    merge_request_depot = dic_depot.copy()

    merge_request_depot.update(dict_split_request_each_depot)

    ouput_split, arr_key_split, arr_key_CutomerCode_split,dict_to_list_id_split,dict_to_list_id_by_cutomer_code_split =convert(merge_request_depot)

    start_LocationCode,end_LocationCode = start_end_locationCode(depot_list_new, vehicle_list_by_depot,dict_to_list_id_split)
    DM_each_depot, TM_each_depot = DM_TM_each_depot(dict_to_list_id_split,DM,TM,arr_key_split)

    weight_each_request, cbm_each_request = cbm_weight_each_request_for_each_depot(dict_split_request_each_depot,depot_list)

    servie_time_depot = service_time_depot(depot_list_new,max_cbm,max_weight)

    cus_depot_working_time, cus_depot_break_time =  working_time_customer_depot_for_each_depot(dict_to_list_id_split,customer_list_by_depot,depot_list,arr_key_split)

    total_service_time, fixed_time = timeService_of_customer_depot(dict_to_list_id_split,arr_key_split,customer_list_by_depot,\
                                    depot_list,weight_each_request,cbm_each_request,servie_time_depot)

    matrix_service_time = f_matrix_service_time(dict_to_list_id_split,total_service_time,fixed_time,depot_list)

    time_matrix_all = TM_each_depot + matrix_service_time
    print(    time_matrix_all , TM_each_depot, matrix_service_time )
    # create vehicle_list, customer_list, depot_list

    vehicle_list_after_split =[]

    for index in range(len(vehicle_list_by_depot)):

        veh = vehicle_list_by_depot[index]

        vehicle_list_after_split.append(VehicleAfterSplit(index, veh.vehicleCode, start_LocationCode[index],end_LocationCode[index],\
                                                        list_time_working_veh[index], veh.cbm, veh.capacity, \
                                                        {"TW_list_free": [[list(list_time_working_veh[index]) ]],
        "TW_used":[    [list_time_working_veh[index][1],list_time_working_veh[index][1]]    ] ,
        "time_list_cus":   [[     ]]}
                                                        ))

    customer_list_after_split = []

    for index in range(len(dict_to_list_id_split)):

        customer_list_after_split.append(CustomerAfterSplit(index, arr_key_split[index], cus_depot_working_time[index],\
                                                        weight_each_request[index], cbm_each_request[index],\
                                                            cus_depot_break_time[index],{"trip":-1 , "vehicle": -1, "time_start":-1}
                                                        ))
        
    number_depot = len(dic_depot)
    depot_list_after_split = copy.deepcopy(customer_list_after_split[:number_depot])

    only_customer_list_after_split = copy.deepcopy(customer_list_after_split[number_depot:])

    #_____________________________________________________________
    #process
    capacity_veh = max_weight
    cbm_veh = max_cbm
    number_of_vehicle = len(vehicle_list_by_depot)
    input_after_split = preprocessing(DM_each_depot,time_matrix_all,cus_depot_working_time,cus_depot_break_time, capacity_veh,\
                    cbm_veh,number_of_vehicle,list_time_working_veh,cbm_each_request,weight_each_request, start_LocationCode, end_LocationCode)

  
    return new_request_each_depot, input_after_split, start_LocationCode, end_LocationCode, vehicle_list_after_split, \
    depot_list_after_split, only_customer_list_after_split, customer_list_after_split, dict_to_list_id_split, arr_key_veh_split,\
     arr_key_split, dict_split_request_each_depot, merge_request_depot, total_service_time, DM_each_depot, weight_each_request, \
     dic_vehicle_after_split, fixed_time,matrix_service_time





In [45]:
# customer_list_by_depot = VDC_new[0]['customer_list_by_depot']
# vehicle_list_by_depot = VDC_new[0]['vehicle_list_by_depot']
# depot_list_new = VDC_new[0]['depot_list_new']

# new_request_each_depot, input_after_split, start_LocationCode,end_LocationCode, vehicle_list_after_split,\
# depot_list_after_split, only_customer_list_after_split, customer_list_after_split, dict_to_list_id_split, arr_key_veh_split, \
# arr_key_split, dict_split_request_each_depot, merge_request_depot, total_service_time, DM_each_depot, weight_each_request, \
# dic_vehicle_after_split, fixed_time = function_split_request(customer_list_by_depot,vehicle_list_by_depot,depot_list_new)


In [46]:
# depot_list_after_split

In [47]:
# dict_to_list_id_split

In [48]:
# total_service_time

## 1. xử lý

In [49]:
def data_preprocessing(

    input_after_split, new_request_each_depot, start_LocationCode,end_LocationCode,
    vehicle_list_after_split, depot_list_after_split, only_customer_list_after_split,
    customer_list_after_split, dict_to_list_id_split, number_of_vehicle, 
    arr_key_veh_split, arr_key_split, dict_split_request_each_depot, merge_request_depot,
    total_service_time, DM_each_depot, weight_each_request, dic_vehicle_after_split,
    fixed_time,vehicle_list_by_depot,matrix_service_time
):

    def create_data_model( input_after_split):

        data = {}

        data['veh_id_list'] = [i for i in range(input_after_split['num_veh'])]
        # data['STARTPOINT'] = STARTPOINT
        # data['ENDPOINT'] = ENDPOINT
        data['capacity'] = input_after_split['capacity_veh']

        data['cbm_veh'] = input_after_split['cbm_veh']

        data['distance_matrix'] = np.copy(input_after_split['distance_matrix'])

        data['time_matrix'] = np.copy(input_after_split['time_matrix'])

        data['time_windows'] =  np.copy(input_after_split['time_windows'])

        data['break_time'] = copy.deepcopy(input_after_split['break_time_cus_depot'])

        data['num_vehicles'] = input_after_split['num_veh']

        data['starts'] = copy.deepcopy(start_LocationCode)

        data['ends'] = copy.deepcopy(end_LocationCode)
        print(data['ends'], data['ends'])

        data['cbm'] = copy.deepcopy(input_after_split['cbm_new'])

        data['weight'] = copy.deepcopy(input_after_split['weight'])

        data['vehicle_capacities'] =  [vehicle_list_after_split[i].capacity for i in data['veh_id_list']]

        data['vehicle_cbm'] =  [vehicle_list_after_split[i].cbm for i in data['veh_id_list']]

        data['time_windows_veh'] = np.array([vehicle_list_after_split[i].tw_list for i in data['veh_id_list']])

        return data



    # Print Solution
    def get_solution(data, manager, routing, solution):

        """Prints solution on console."""
        print(f'Objective: {solution.ObjectiveValue()}')
        result  ={"assigned_vehicles":{ "index_veh":[], "plan_output":[], "TW_cus":[] }, 
                "free_vehicles":{ "index_veh":[], "time_window_veh":[] }, 
                "unassigned_request":[]}
        max_route_distance = 0

        for vehicle_id in range(data['num_vehicles']):

            index = routing.Start(vehicle_id) # xác định đỉnh bắt đầu của vehicle_index
            plan_output = 'Route for vehicle {}:\n'.format(vehicle_id) # lưu thông tin tuyến đường của vehicle_id
            time_dimension = routing.GetDimensionOrDie('Time')
            time_var = time_dimension.CumulVar(index)
            total_time = 0
            route_distance = 0 # tính khoảng cách tuyến đường của vehicle_id
            route = []
            TW_cus =[]

            route.append(manager.IndexToNode(index))
            TW_cus.append( [solution.Min(time_var), solution.Max(time_var)])
            
            while not routing.IsEnd(index):
                
                time_var = time_dimension.CumulVar(index)
                plan_output += ' {} -> '.format(manager.IndexToNode(index)) # thêm thông tin đỉnh hiện tại
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                time_var = time_dimension.CumulVar(index)
                route_distance += routing.GetArcCostForVehicle(
                    previous_index, index, vehicle_id)
                
                route.append(manager.IndexToNode(index))   
                TW_cus.append( [solution.Min(time_var),    
                                solution.Max(time_var)]) 
            
            plan_output += '{}\n'.format(manager.IndexToNode(index))
            plan_output += 'Distance of the route: {}m\n'.format(route_distance)

            if route_distance > 0:
                result['assigned_vehicles']["index_veh"].append(vehicle_id)
                result['assigned_vehicles']["plan_output"].append(route)
                result['assigned_vehicles']["TW_cus"].append(TW_cus)
            else:
                result['free_vehicles']["index_veh"].append(vehicle_id)
            
            # print(plan_output)
            max_route_distance = max(route_distance, max_route_distance)

        for i in range(len(data["distance_matrix"])):
            found =False
            for a in result['assigned_vehicles']['plan_output']:  
                if i in a:
                    found = True
                    break
            if not found:
                result['unassigned_request'].append(i)

        return  result



    # Add Constrain
    def add_time_windows_constraint(routing,manager, data):

        # Create and register a transit callback.
        def time_callback(from_index, to_index):
            """Returns the travel time between the two nodes."""
            # Convert from routing variable Index to time matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['time_matrix'][from_node][to_node]
        time_transit_callback_index = routing.RegisterTransitCallback(time_callback)

        def distance_callback(from_index, to_index):
            """Returns the distance between the two nodes."""
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]
        distance_transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        def demand_callback(from_index):
            """Returns the demand of the node."""
            # Convert from routing variable Index to demands NodeIndex.
            from_node = manager.IndexToNode(from_index)
            return data['cbm'][from_node]
        demand_cbm_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

        def demand_callback(from_index):
            """Returns the demand of the node."""
            # Convert from routing variable Index to demands NodeIndex.
            from_node = manager.IndexToNode(from_index)
            return data['weight'][from_node]
        demand_weight_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(distance_transit_callback_index)

        max_time_per_veh = np.zeros((data['num_vehicles'],),np.int_)

        for vehicle_id in range(data['num_vehicles']): 
            depot_end_index = data['ends'][vehicle_id]
            time_depot = data['time_windows'][depot_end_index][1].item()
            time_veh = data['time_windows_veh'][vehicle_id][1].item()
            max_time_per_veh[vehicle_id] = min(time_depot,time_veh)
        
        # Add Time Windows constraint.
        time = 'Time'
        routing.AddDimensionWithVehicleCapacity(
            time_transit_callback_index,
            300000,  # allow waiting time
            max_time_per_veh,  # maximum time per vehicle
            False,  # Don't force start cumul to zero.
            time)
        time_dimension = routing.GetDimensionOrDie(time)

        # Add time window constraints for each location except depot.
        # for location_idx, time_window in enumerate(data['time_windows']):
        for location_idx in range(len(data['time_windows'])): 

            if location_idx in data['starts'] or location_idx in data['ends']:
                continue
            
            index = manager.NodeToIndex(location_idx)
            time_dimension.CumulVar(index).SetRange(data['time_windows'][location_idx,0].item(), data['time_windows'][location_idx,1].item())

            print("break_time", location_idx,data['break_time'] )
            break_time_list_at_customer = data['break_time'][location_idx]
            print("break_time_list_at_customer", break_time_list_at_customer)
            for break_interval in break_time_list_at_customer:
                print(break_interval)
                break_start = break_interval[0] #convert to minute
                break_end = break_interval[1]
                time_dimension.CumulVar(index).RemoveInterval(break_start, break_end)
            # time_dimension.CumulVar(index).RemoveInterval(break_start, break_end)

        # Add time window constraints for each vehicle start node.
        for vehicle_id in range(data['num_vehicles']): 

            depot_idx = data['starts'][vehicle_id]

            index = routing.Start(vehicle_id)
        
            time_dimension.CumulVar(index).SetRange(
                max(data['time_windows'][depot_idx][0].item(),data['time_windows_veh'][vehicle_id][0].item() ),
                min(data['time_windows'][depot_idx][1].item(), data['time_windows_veh'][vehicle_id][1].item() )
            )
            
        # Instantiate route start and end times to produce feasible times.
        for i in range(data['num_vehicles']):
        #     routing.AddVariableMaximizedByFinalizer(
        #         time_dimension.CumulVar(routing.Start(i)))
            routing.AddVariableMaximizedByFinalizer( time_dimension.CumulVar(routing.End(i)))

        # Add Capacity constraint.

        routing.AddDimensionWithVehicleCapacity(
            demand_cbm_callback_index,
            0,  # null capacity slack
            data['vehicle_cbm'],  # vehicle maximum capacities
            True,  # start cumul to zero
            'cbm')

        routing.AddDimensionWithVehicleCapacity(
            demand_weight_callback_index,
            0,  # null capacity slack
            data['vehicle_capacities'],  # vehicle maximum capacities
            True,  # start cumul to zero
            'Capacity')



    # Solver
    def solver(data):
        """Solve the VRP with time windows."""
        # Instantiate the data problem.
        # Create the routing index manager.
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'],  data['starts'],data['ends'])

        # Create Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        add_time_windows_constraint(routing, manager,data)
        
        # Allow to drop nodes.
        # penalty = 20000000
        number_depot = len(depot_list_after_split)

        for node in range(number_depot,len(data['time_matrix'])):
            time_waste =   data['time_matrix'][ data['starts'][0],node]+  data['time_matrix'][node, data['ends'][0]] 
            x= round(data['weight'][node]*2_000_000_000/  (10+ time_waste))
            routing.AddDisjunction([manager.NodeToIndex(node)],x )

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()


        search_parameters.first_solution_strategy = ( routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_INSERTION)

        # Solve the problem.
        print("start to solve")
        solution = routing.SolveWithParameters(search_parameters)
        
        # Print solution on console.
        if solution:
            if routing.status( ) == 1:
                print("Solver status: ", routing.status( ))
                return get_solution(data, manager, routing, solution)
            else:
                print("Solver status: ", routing.status( ))
                return {"assigned_vehicles":{
            "index_veh":[],
            "plan_output":[],
            "TW_cus":[]
        }, "free_vehicles":{
            "index_veh":[],
            "time_window_veh":[]
        }, "unassigned_request":[cus.id for cus in only_customer_list_after_split]}
        else:

            print("No solution!")
            return    {"assigned_vehicles":{
            "index_veh":[],
            "plan_output":[],
            "TW_cus":[]
        }, "free_vehicles":{
            "index_veh":[],
            "time_window_veh":[]
        }, "unassigned_request":[cus.id for cus in only_customer_list_after_split]}



    # Tranform index of customer and vehicle
    def transform_index_cus(input,result):

        result_new = copy.deepcopy(result)
        list_of_index_depot_and_customers =  input[1] + input[0]
        # print(list_of_index_depot_and_customers, type(list_of_index_depot_and_customers))

        plan_output_new = [[list_of_index_depot_and_customers[point] for point in trip  ] for trip in result["assigned_vehicles"]['plan_output']]

        unassigned_request_new = [list_of_index_depot_and_customers[point] for point in result["unassigned_request"]]

        index_veh_new = [input[2][veh]   for veh in result["assigned_vehicles"]['index_veh']]

        result_new["assigned_vehicles"]['plan_output'] = copy.deepcopy(plan_output_new)

        result_new["unassigned_request"] = copy.deepcopy(unassigned_request_new)

        result_new["assigned_vehicles"]['index_veh'] = copy.deepcopy(index_veh_new)

        return result_new



    # hàm này đang không được dùng ...
    # Prepare data for the next round
    def extract_time_from_result(result_i):
        return [result_i[0][1], result_i[-1][1]]


    def remove_an_interval_from_tw( interval, tw):
        if  interval[0]   - tw[0]==0:
            return [[ interval[1], tw[1]]]
        if - interval[1]   + tw[1]==0:
            return [ [ tw[0], interval[0]]]
        return  [[ tw[0], interval[0]] , [ interval[1], tw[1]]]


    def time_min_from_depot_cus_depot(list_customer, start, end,TM_each_depot):
        if len(list_customer)==0:
            return 1000000000000000
        print([ TM_each_depot[start, node]+ TM_each_depot[node, end] for node in list_customer ])
        return min( [ TM_each_depot[start, node]+ TM_each_depot[node, end] for node in list_customer ])


    def repair_vehicle(new_data, data,input_after_split, result):

        # used_time= [ ]


        new_data['veh_id_list']=[]

        start_depot_point = [depot.id for depot in depot_list_after_split]
        
        buffer_time =  11111111111111111111111111111111

        for start in start_depot_point:
            print("start",start)
            a = time_min_from_depot_cus_depot( result['unassigned_request'],start,start,input_after_split['time_matrix'])
            print("test _min",a)
            if a < buffer_time:

                buffer_time = a

       
        count_veh = 0

        for index in range(len(result["assigned_vehicles"]["index_veh"])):

            veh_id = result["assigned_vehicles"]["index_veh"][index]

            tw_free_of_veh_id = result["assigned_vehicles"]["TW_cus"][index][0]

            print("check tw", index, tw_free_of_veh_id,buffer_time)
            if tw_free_of_veh_id[1] - tw_free_of_veh_id[0] >= buffer_time:

                

                new_data['veh_id_list'].append(veh_id)

                new_data['time_windows_veh'][count_veh][0],new_data['time_windows_veh'][count_veh][1] = tw_free_of_veh_id[0], tw_free_of_veh_id[1]      
                count_veh += 1

        
        new_data['num_vehicles'] = count_veh



    def prepare_next_loop_data(data, input_after_split,result):

        # xu ly xe
        new_data = copy.deepcopy(data)
        repair_vehicle(new_data, data,input_after_split, result)
        new_data['capacity'] = data['capacity']
        new_data['cbm_veh'] = data['cbm_veh']
        new_data['vehicle_capacities'] = [data['capacity']]*new_data['num_vehicles']
        new_data['vehicle_cbm'] = [data['cbm_veh']]*new_data['num_vehicles']

        # xu ly depot 

        # new_data['STARTPOINT']=STARTPOINT

        # new_data['ENDPOINT']=ENDPOINT


        
        new_data['ends']= [vehicle_list_after_split[i].endLocationCode for i in new_data['veh_id_list']]

        print("new_data['ends']", new_data['ends'])
        
        new_data['starts']= [vehicle_list_after_split[i].startLocationCode for i in new_data['veh_id_list']]
        
        print("new_data['starts']", new_data['starts'])
        
        start_depot_point = [depot.id for depot in depot_list_after_split]

        input_new = [] # include unassigned request + STARTPOINT_index + list of vehicle index
        input_new.append( result['unassigned_request'])
        input_new.append(start_depot_point)
        input_new.append(new_data['veh_id_list'])


        # print("input_new",input_new)

    

        index_list =start_depot_point + result['unassigned_request'] 

        # index_list là id cua cac node bi drop
        # print("id cua cac node bi drop", index_list)

        
        new_DM= np.empty((len(index_list), len(index_list)), np.int_   )
        new_TM = np.empty((len(index_list), len(index_list)), np.int_   )
        new_TW_cus = np.empty((len(index_list),2), np.int_   )

        for i in range(len(index_list)):
            for j in range(len(index_list)):
                new_DM[i,j] = input_after_split['distance_matrix'][index_list[i] ][index_list[j]]
                new_TM[i,j] = input_after_split['time_matrix'][index_list[i] ][index_list[j]]

        new_data['distance_matrix'] = np.copy(new_DM)
        
        new_data['time_matrix'] = np.copy(new_TM)

        new_data["time_windows"] = np.array([customer_list_after_split[i].tw_list for i in index_list])



        new_data['break_time'] =[customer_list_after_split[i].break_time_customer for i in index_list]

        new_data['cbm' ] = [customer_list_after_split[i].cbm for i in index_list]

        new_data['weight'] = [customer_list_after_split[i].cbm for i in index_list]

        # new_data['convert_from_split_to_origin_node']= data['convert_from_split_to_origin_node']
        
   
        # new_data['original_input'] = copy.deepcopy(data['original_input'])
        
        return new_data, index_list,input_new 



    # Update customer, vehicle list
    def update_to_cus_list(list_result, vehicle_list_after_split,customer_list_after_split,new_data):
        
        TW_veh = {}
    
        for index_trip,result in enumerate(list_result):
            
            for index,trip in enumerate(result['assigned_vehicles']['plan_output']):
        
                index_cus=0
                for index_cus,i in enumerate(trip):

                    customer_list_after_split[i].state["trip"] = index_trip
                    customer_list_after_split[i].state["vehicle"] = result['assigned_vehicles']['index_veh'][index]
                    customer_list_after_split[i].state["time_start"] = result['assigned_vehicles']['TW_cus'][index][index_cus][1]
                    customer_list_after_split[i].is_visited = result['assigned_vehicles']['index_veh'][index]

        result =list_result[-1]
        
        for index in range(len(result['assigned_vehicles']['index_veh'])):
        # for index in result['assigned_vehicles']['index_veh']:

            value_veh = result['assigned_vehicles']['index_veh'][index]

            TW_use_this_time = [result['assigned_vehicles']['TW_cus'][index][0][1], result['assigned_vehicles']['TW_cus'][index][-1][1]]

            vehicle_list_after_split[value_veh].state['TW_used'].append(TW_use_this_time)

            vehicle_list_after_split[value_veh].state['time_list_cus'].append([result['assigned_vehicles']['TW_cus'][index][index_cus][1] for index_cus in range(len(result['assigned_vehicles']['TW_cus'][index]))])

            # print(a)
            # print(new_data['time_windows_veh'][index])
            vehicle_list_after_split[value_veh].state['TW_list_free'].append(remove_an_interval_from_tw( TW_use_this_time, new_data['time_windows_veh'][index]))



    # Visulize
    def visualize_vehicle_list(input,vehicle_list_after_split):
        figure(figsize=(8, 6), dpi=120)

    # veh_id_list
        for k in range(len(input[2])):
            i =  input[2][k]
            state= vehicle_list_after_split[i].state
            # print(k,"k")
            for j in range(len(state['TW_used'])):
                interval = [ state['TW_used'][j][0], state['TW_used'][j][1]]
                plt.plot( [interval[0]/100, interval[1]/100], [k-j/40,k-j/40], lw=1)
                plt.plot( interval[0]/100, k-j/40, 'bo', ms=1)
                plt.plot( interval[1]/100,k-j/40, 'bo', ms=1)
                # print([interval[0], interval[1]],)
                # print(interval)
            
          
            plt.plot( [vehicle_list_after_split[i].tw_list[0]/100, vehicle_list_after_split[i].tw_list[1]/100], [k+0.05,k+0.05], lw=1.5)

            # plt.plot( [interval[0], interval[1]], [i,i], lw=3)
        plt.show()



    def visualize_customer_list(input,customer_list_after_split):
        figure(figsize=(8, 6), dpi=120)

        plt.ylim([-1.2, len(vehicle_list_after_split)-1])
        plt.xticks(range(len(customer_list_after_split)))
        plt.yticks(range(-1, len(vehicle_list_after_split)))

        for j in range(len(input[2])):
            i=input[2][j]
            if customer_list_after_split[i].state['vehicle'] ==-1:
                plt.plot(i, customer_list_after_split[i].state['vehicle'],"ro")
            else:
                plt.plot(i, customer_list_after_split[i].state['vehicle'],"bo")
            plt.plot( [ i,i], [-1, len(vehicle_list_after_split)], 'k', linewidth=0.1)
        plt.show()



    def visulize_tw(list_result,input_after_split):

        # figure(figsize=(8, 6), dpi=120)
        for result in list_result:
    
            trip_info =[]

            for index in range(len(result["assigned_vehicles"]['TW_cus'])):
                
                trip_TW = result["assigned_vehicles"]['TW_cus'][index]

                for tw_cus_index in range(len(trip_TW)):
                    
                    plt.plot(tw_cus_index,trip_TW[tw_cus_index][0], "bo" )
                    plt.plot(tw_cus_index,trip_TW[tw_cus_index][1],"bo" )
                    plt.plot( [tw_cus_index, tw_cus_index], input_after_split['time_windows'][result["assigned_vehicles"]['plan_output'][index][tw_cus_index]], "r" )
                    
                    plt.text(tw_cus_index,-2 , "node  "+str(result["assigned_vehicles"]['plan_output'][index][tw_cus_index]))
                    
                    # print(result["assigned_vehicles"]['index_veh'][index])
                plt.plot( [0,len(trip_TW)-1], [input_after_split['time_windows_veh'][result["assigned_vehicles"]['index_veh'][index]][1], input_after_split['time_windows_veh'][result["assigned_vehicles"]['index_veh'][index]][1]] , "k" )
                plt.plot( [0,len(trip_TW)-1], [input_after_split['time_windows_veh'][result["assigned_vehicles"]['index_veh'][index]][0], input_after_split['time_windows_veh'][result["assigned_vehicles"]['index_veh'][index]][0]] , "k" )
                plt.text( 0, -10, "trip  "+str(index))
                plt.show()



    # Main

    number_depot = len(depot_list_after_split)

    number_request_depot = len(dict_to_list_id_split)

    numer_customer = number_request_depot - number_depot

    customer_id_list  = list(range(number_depot,number_request_depot ))
    depot_id_list  = list(range(number_depot))
    veh_id_list  =list(range(0,number_of_vehicle))

    input_id= [customer_id_list, depot_id_list,veh_id_list]



    # time1 =time.time()
    def main(input_id,input_after_split):

        data = create_data_model(input_after_split)

        new_data = copy.deepcopy(data)
    
        result= solver(data)
        # print(time.time()-time1 , "1 ")

        new_result = transform_index_cus(input_id, result)
        # print(time.time()-time1 , "2 ")

        list_result = [result]

        print(result)
        # print("newresult",new_result)
        update_to_cus_list(list_result, vehicle_list_after_split,customer_list_after_split,new_data)
        # print(time.time()-time1 , "3 ")

        # visualize_customer_list(input_id,customer_list_after_split)
            
        # visualize_vehicle_list(input_id,vehicle_list)

        # visulize_tw(list_result,input_after_split)

      

        k=10
        while k!=0:

            print(  k, "0 ")
            if len(new_result['assigned_vehicles']['index_veh']) ==0:
                print("drop all request")
                break
            

            new_data, index_list, input_new =  prepare_next_loop_data(new_data,input_after_split, new_result)
            print( k, "1 ")

            print("started to solved with",len(new_result['unassigned_request']), "cus ", new_data["num_vehicles"], "veh")
            
            if len(new_result['unassigned_request']) <2  or new_data["num_vehicles"] ==0:
                break

            new_result = solver(new_data)
            print( k, "2 ")
        
            # print(time.time()- time1, "s ")


            new_result = transform_index_cus(input_new,  new_result)
            # print(time.time()-time1 , k, "3 ")
            print(new_result)

            list_result.append(new_result)
            if len(new_result['assigned_vehicles']['index_veh']) ==0:
                print("drop all request")
                break
            
            update_to_cus_list(list_result, vehicle_list_after_split,customer_list_after_split,new_data)
            # print(time.time()-time1 , k, "4 ")

            # print("vehicle_list",vehicle_list)
            
            # visualize_customer_list(input_id,customer_list_after_split)
            
            # visualize_vehicle_list(input_id,vehicle_list_after_split)

            # visulize_tw(list_result,input_after_split)

            print("----------------------------------")
            k -=1
    
        return list_result
        
    list_result = main(input_id,input_after_split)

    print("list_reult ", list_result)
    # ------>update trạng thái vào cusstomer list gốc để đễ keeep track
    # {'assigned_vehicles': {'index_veh': [0, 1], 'plan_output': [[0, 3, 4, 9, 1], [0, 5, 10, 15, 1]]}, 'free_vehicles': [], 'unassigned_request': [13, 17, 18, 20]}

    def convert_list_result_to_origin(list_result):
        
        reuslt_with_index_origin = copy.deepcopy(list_result)

        for result in reuslt_with_index_origin:
            
            index_veh = result['assigned_vehicles']['index_veh']
            
            result['assigned_vehicles']['index_veh'] = [arr_key_veh_split[veh] for veh in index_veh]
            
            # print(result['assigned_vehicles']['index_veh'])

            plan_output = result['assigned_vehicles']['plan_output']

            plan_output_with_index_origin = []
            
            for trip in plan_output:
                
                trip_index_new = [arr_key_split[i] for i in trip]
                
                plan_output_with_index_origin.append(trip_index_new)

            result['assigned_vehicles']['plan_output'] = copy.deepcopy(plan_output_with_index_origin)
            
            result['free_vehicles'] = [arr_key_veh_split[veh] for veh in result['free_vehicles']['index_veh']]
        
            unassigned_request =  result['unassigned_request']
            result['unassigned_request'] = [arr_key_split[i] for i in unassigned_request]
        
        return reuslt_with_index_origin

    reuslt_with_index_origin = convert_list_result_to_origin(list_result)
    print("reuslt_with_index_origin", reuslt_with_index_origin)
    def merge_item_each_request(dict_split_request):

        merge_item_each_request ={}
        for index_requets, val in dict_split_request.items():

            all_item_each_request = dict_split_request[index_requets]['items']
            
            for index_item in range(len(all_item_each_request)):

                item = all_item_each_request[index_item]

                dict_split_request[index_requets]['items'][index_item]['quantity'] = 1
        
       

        for index_requets, val in dict_split_request.items():

            all_item_each_request = dict_split_request[index_requets]['items']
     
            groups = {}
            for item in all_item_each_request:
                if item['itemCode'] not in groups:
                    groups[item['itemCode']] = {
                        'itemCode': item['itemCode'],
                        'quantity': item['quantity'],
                        
                        'weight': item['weight'],
                        'cbm': item['cbm'],
                        'quantityPerPallet': item['quantityPerPallet'],
                        'size': item['size'],
                        'iType':item['iType'],
                        'orderCode': item['orderCode']
                    }
                else:
                    groups[item['itemCode']]['quantity'] += 1
                    groups[item['itemCode']]['weight'] += item['weight']
                    groups[item['itemCode']]['cbm'] += item['cbm']
                    
            merge_item_each_request[index_requets] = list(groups.values())
            
        return merge_item_each_request
            
    merge_item_each_request = merge_item_each_request(dict_split_request_each_depot)

 

    def merge_item_each_customer(customer_depot_ordercode):
        # merge_item_each_customer={}
        group_origin_request = {}
        
        for index_requests, val in customer_depot_ordercode.items():
            
            
            all_items_in_each_depot_code = customer_depot_ordercode[index_requests]

            key_dict_new = index_requests[:2] + (index_requests[3],)

            group_origin_request[key_dict_new] = []

        for index_requests, val in customer_depot_ordercode.items():
            
            
            all_items_in_each_depot_code = customer_depot_ordercode[index_requests]

            key_dict_new = index_requests[:2] + (index_requests[3],)

            group_origin_request[key_dict_new].append(val)

        return group_origin_request
    

    def get_more_infor_request(key,details_list,trip_loop,key_index):

        print("detail_list_before", details_list)
        for veh in vehicle_list_by_depot:
            if key == veh.vehicleCode:
                startLocationCode = veh.startLocationCode
                start_Code = (startLocationCode,)
        print("startCode", start_Code)        
        index_startLocationCode = dic_location[startLocationCode]

        startLocationCode_depot = details_list[0][0]
        index_startLocationCode_depot = dic_location[startLocationCode_depot]

        details_list.insert(0,start_Code)
        print("detail_lisst_new", details_list)
        elements = []
        distance_cumul = 0

        weight_each_item_loaded = 0

        cbm_each_item_loaded = 0

        for index_node in range(len(details_list)):
            value_node =  details_list[index_node]
            infor_each_elements = {}
            
            
            if index_node == 0:

                infor_each_elements["location_code"] = value_node[0]
                infor_each_elements["cd_code"] = value_node[0]
                infor_each_elements['distance'] = 0
                infor_each_elements["weight_load"] = 0
                infor_each_elements["cbm_load"] = 0
                
                leaving_time= reuslt_with_index_origin[trip_loop]['assigned_vehicles']['TW_cus'][key_index][index_node][1] + date_start

                arrival_time = leaving_time - TM[index_startLocationCode,index_startLocationCode_depot]
            
                infor_each_elements["arrival_time"] = dt.fromtimestamp(arrival_time).strftime("%Y-%m-%d %H:%M:%S") 
                infor_each_elements["leaving_time"] = dt.fromtimestamp(arrival_time).strftime("%Y-%m-%d %H:%M:%S")


                infor_each_elements["location_type"] = "STATION"
                infor_each_elements["items"] = []
        
            elif index_node == 1:

                all_value_node = merge_request_depot[value_node]
                infor_each_elements["location_code"] = all_value_node['depotCode']
                infor_each_elements["cd_code"] = all_value_node['depotCode']
                infor_each_elements['distance'] = 0

                total_weight_load = 0
    
                total_cbm_load = 0

                for index_node_s in range(1,len(details_list)-1):

                    value_node_s =  details_list[index_node_s]

                    total_weight_load += input_after_split['weight'][dict_to_list_id_split[value_node_s]]

                    total_cbm_load += input_after_split['cbm_new'][dict_to_list_id_split[value_node_s]]

                infor_each_elements["weight_load"] =  int(total_weight_load)
                infor_each_elements["cbm_load"] = int(total_cbm_load)

                # print(reuslt_with_index_origin[trip_loop]['assigned_vehicles']['TW_cus'][key_index][index_node-1][1] )

                arrival_time= reuslt_with_index_origin[trip_loop]['assigned_vehicles']['TW_cus'][key_index][index_node-1][1] + date_start

                # index_origin = dict_to_list_id_split[value_node]

                
                value_node_end =  details_list[index_node + 1]

                index_origin_start = dict_to_list_id_split[value_node]
                
                index_origin_end = dict_to_list_id_split[value_node_end]

                print("check_time_depot", value_node,value_node_end, matrix_service_time[index_origin_start,index_origin_end], total_service_time[index_origin_start] )

                # leaving_time = arrival_time + matrix_service_time[index_origin_start,index_origin_end] - total_service_time[index_origin_end]
                leaving_time = arrival_time + total_service_time[index_origin_start] + fixed_time[index_origin_start]
                # print("index_origin", total_service_time[index_origin] )
                
                infor_each_elements["arrival_time"] = dt.fromtimestamp(arrival_time).strftime("%Y-%m-%d %H:%M:%S")
                infor_each_elements["leaving_time"] = dt.fromtimestamp(leaving_time).strftime("%Y-%m-%d %H:%M:%S")
               
                infor_each_elements["location_type"] = "DEPOT"
                infor_each_elements["items"] = []

                for index_node_x in range(2,len(details_list)-1):
                    print("details_list",details_list)
                    print("index_node", index_node_x)
                    value_node_x =  details_list[index_node_x]

                    all_value_items = merge_item_each_request[value_node_x]
                    
                    for each_item in all_value_items:
                        item ={}
 

                        item['item_code'] = each_item['itemCode']

                        item['quantity'] = each_item['quantity']

                        item['weight'] = each_item['weight']

                        item['cbm'] = each_item['cbm']

                        item['size'] = each_item['size']
                    
                        infor_each_elements["items"].append(item)

            elif index_node == len(details_list) -1:

                all_value_node = merge_request_depot[value_node]

                infor_each_elements["location_code"] = value_node[0]
                infor_each_elements["cd_code"] = value_node[0]

                value_node_start =  details_list[index_node -1]
                index_origin_start = dict_to_list_id_split[value_node_start]
                index_origin_end = dict_to_list_id_split[value_node]

                infor_each_elements['distance'] = int(distance_cumul + DM_each_depot[index_origin_start,index_origin_end])

                print("distance", infor_each_elements['distance'] , type(infor_each_elements['distance']))

                weight_each_item_loaded += input_after_split['weight'][dict_to_list_id_split[value_node]]

                cbm_each_item_loaded += input_after_split['cbm_new'][dict_to_list_id_split[value_node]]

                infor_each_elements["weight_load"] = int(total_weight_load - weight_each_item_loaded)

                infor_each_elements["cbm_load"] = int(total_cbm_load - cbm_each_item_loaded)


                # print(reuslt_with_index_origin[trip_loop]['assigned_vehicles']['TW_cus'][key_index][index_node-1][1] )

                leaving_time = reuslt_with_index_origin[trip_loop]['assigned_vehicles']['TW_cus'][key_index][index_node-1][1]  + date_start
                index_origin = dict_to_list_id_split[value_node]
                
                arrival_time = leaving_time - fixed_time[index_origin] 

                print("check time:","index_origin",index_origin,fixed_time[index_origin]  )

                infor_each_elements["arrival_time"] = dt.fromtimestamp(arrival_time).strftime("%Y-%m-%d %H:%M:%S")
                infor_each_elements["leaving_time"] = dt.fromtimestamp(leaving_time).strftime("%Y-%m-%d %H:%M:%S")

                
                infor_each_elements["location_type"] = dic_location_type[value_node[0]][0]
                infor_each_elements["items"] = []
            
            else: 
                all_value_node = merge_request_depot[value_node]
                index_request = value_node[:2]
                all_value_node_request = new_request_each_depot[index_request]
                infor_each_elements["location_code"] = index_request[0]
                infor_each_elements["cd_code"] = all_value_node_request[0]['customerCode']

                value_node_start =  details_list[index_node -1]
                index_origin_start = dict_to_list_id_split[value_node_start]
                index_origin_end = dict_to_list_id_split[value_node]

                infor_each_elements['distance'] = int(distance_cumul + DM_each_depot[index_origin_start,index_origin_end])
                distance_cumul = distance_cumul + DM_each_depot[index_origin_start,index_origin_end]

                weight_each_item_loaded =  weight_each_item_loaded + input_after_split['weight'][dict_to_list_id_split[value_node]]

                cbm_each_item_loaded = cbm_each_item_loaded + input_after_split['cbm_new'] [dict_to_list_id_split[value_node]]

                infor_each_elements["weight_load"] = int(total_weight_load - weight_each_item_loaded)
 
                infor_each_elements["cbm_load"] = int(total_cbm_load - cbm_each_item_loaded)

                # infor_each_elements["weight_load"] = input_after_split['weight'][dict_to_list_id_split[value_node]]
                # infor_each_elements["cbm_load"] = input_after_split['cbm_new'] [dict_to_list_id_split[value_node]]

                leaving_time = reuslt_with_index_origin[trip_loop]['assigned_vehicles']['TW_cus'][key_index][index_node-1][1]  + date_start

                index_origin = dict_to_list_id_split[value_node]
                

                arrival_time = leaving_time - total_service_time[index_origin] - fixed_time[index_origin]

                infor_each_elements["arrival_time"] = dt.fromtimestamp(arrival_time).strftime("%Y-%m-%d %H:%M:%S")
                infor_each_elements["leaving_time"] = dt.fromtimestamp(leaving_time).strftime("%Y-%m-%d %H:%M:%S")
                print("_____________________________________________________")
                print("value_node",index_node,value_node,total_service_time[index_origin],infor_each_elements["arrival_time"],infor_each_elements["leaving_time"] )

                infor_each_elements["location_type"] = "CUSTOMER"           
                infor_each_elements["items"] = []

                

                all_value_items = merge_item_each_request[value_node]
                
                for each_item in all_value_items:

                    item ={}
                    item['item_code'] = each_item['itemCode']
                    item['quantity'] = each_item['quantity']
                    item['weight'] = each_item['weight']
                    item['cbm'] = each_item['cbm']
                    item['size'] = each_item['size']
                    infor_each_elements["items"].append(item)

            elements.append(infor_each_elements)
        
        return elements


    # infor = get_more_infor_request(details_list,0,0)
    # infor


    def preprocessing_output(reuslt_with_index_origin):

        dict_veh ={}

        for trip_loop in range(len(reuslt_with_index_origin)):

            trip_value = reuslt_with_index_origin[trip_loop]

            keys_veh = trip_value["assigned_vehicles"]["index_veh"]

            for key in keys_veh:

                dict_veh[key] =[]

        for trip_loop in range(len(reuslt_with_index_origin)):

            trip_value = reuslt_with_index_origin[trip_loop]
            
            keys_veh = trip_value["assigned_vehicles"]["index_veh"]

            for key_index in range(len(keys_veh)):
                key = keys_veh[key_index]
                key_veh_value = {}
                details_list = trip_value["assigned_vehicles"]['plan_output'][key_index]
                # print(details_list)

                key_veh_value['plan_output'] = get_more_infor_request(key,details_list,trip_loop,key_index)
                key_veh_value['TW_cus'] =  trip_value["assigned_vehicles"]['TW_cus'][key_index]
            
                dict_veh[key].append(key_veh_value)
        

        return dict_veh

    dict_veh = preprocessing_output(reuslt_with_index_origin)

    id_depot = [depot.customerCode for depot in depot_list_after_split] 
    # id_depot

    def unscheduled_requests_1(reuslt_with_index_origin):
        
        # unassigned_request =reuslt_with_index_origin[-1]['unassigned_request']
        id_depot = [depot.customerCode for depot in depot_list_after_split] 
        unassigned_request =[]
        for index in reuslt_with_index_origin[-1]['unassigned_request']:
            if index not in id_depot:
                unassigned_request.append(index)

        dict_index_request_value={}

        for index_request in range(len(unassigned_request)):
            
            id_request = unassigned_request[index_request]

            val_request = merge_item_each_request[id_request]

            dict_index_request_value[id_request] = val_request
        
        dict_index_request_value_orderCode = {}
        
        for key, val in dict_index_request_value.items():
            
            for item in dict_index_request_value[key]:

                itemCode = item['orderCode']

                key_new = key + (itemCode,)

                dict_index_request_value_orderCode[key_new] =[]
            
            for item in dict_index_request_value[key]:

                itemCode = item['orderCode']

                key_new = key + (itemCode,)

                dict_index_request_value_orderCode[key_new].append(item)

        return dict_index_request_value_orderCode

    dict_index_request_value_orderCode = unscheduled_requests_1(reuslt_with_index_origin)

    dict_index_unassigned_request_value_orderCode = merge_item_each_customer(dict_index_request_value_orderCode)



    def sum_val_same_item(dict_index_unassigned_request_value_orderCode):
    
        merge_value_item = {}

        for key_orderCode, val in  dict_index_unassigned_request_value_orderCode.items():
            
            groups ={}
            
            for item_oderCode in dict_index_unassigned_request_value_orderCode[key_orderCode]:

                for item in item_oderCode:
                    if item['itemCode'] not in groups:
                        groups[item['itemCode']] = {
                            'itemCode': item['itemCode'],
                            'quantity': item['quantity'],
                            
                            'weight': item['weight'],
                            'cbm': item['cbm'],
                            'quantityPerPallet': item['quantityPerPallet'],
                            'size': item['size'],
                            'iType':item['iType'],
                            'orderCode': item['orderCode']
                        }
                    else:
                        groups[item['itemCode']]['quantity'] += item['quantity']
                        groups[item['itemCode']]['weight'] += item['weight']
                        groups[item['itemCode']]['cbm'] += item['cbm']

            merge_value_item[key_orderCode] = list(groups.values())

        return merge_value_item

    merge_value_item = sum_val_same_item(dict_index_unassigned_request_value_orderCode)



    def unscheduled_requests_output(merge_value_item):

        unscheduled_requests =[]
        
        for key_id_req, val in merge_value_item.items():

            id_req_cus_depot = key_id_req[:2]

            all_infor_req = new_request_each_depot[id_req_cus_depot]          

            unscheduled_request={}

            unscheduled_request['order_code'] = key_id_req[2]

            unscheduled_request['depot_code'] = all_infor_req[0]['depotCode']

            unscheduled_request['pickup_location_code'] = key_id_req[1]
                
            unscheduled_request['customer_code'] = all_infor_req[0]['customerCode']

            unscheduled_request['delivery_location_code'] = key_id_req[0]

            unscheduled_request['items'] =[]

            for each_item in merge_value_item[key_id_req]:

                item ={}

                item['item_code'] = each_item['itemCode']

                item['quantity'] = each_item['quantity']

                item['weight'] = each_item['weight']

                item['cbm'] = each_item['cbm']

                item['size'] = each_item['size']

                unscheduled_request['items'].append(item)
            
            weight =0
            for item in unscheduled_request['items']:
                weight += item['weight']

            unscheduled_request['weight'] = weight

            cbm =0
            for item in unscheduled_request['items']:
                weight += item['cbm']

            unscheduled_request['cbm'] = cbm

            unscheduled_requests.append(unscheduled_request)
        return unscheduled_requests

    unscheduled_requests = unscheduled_requests_output(merge_value_item)



    def convert_to_output(dict_veh):

    
        output = {}
        solutions =[]

        solution ={ "routes":[], "additional_locations": [  ], "unscheduled_requests": [  ]}

        for key_veh, val in dict_veh.items():

            for route_value in val:

                route = {}

                all_value_of_each_vehicle = dic_vehicle_after_split[key_veh][0]

                route['vehicle_code'] =key_veh

                route['romooc_code'] = all_value_of_each_vehicle['romoocCode']

                route['vehicle_weight'] = all_value_of_each_vehicle['capacity']

                route['vehicle_cbm'] = all_value_of_each_vehicle['cbm']

                # total_weight_load = 0

                # total_cbm_load = 0

                for element in route_value['plan_output']:
                    
                    if element['location_type'] == "DEPOT":
                        
                        total_weight_load = element['weight_load']

                        total_cbm_load = element['cbm_load']

                        break

                route['total_weight_load'] =  total_weight_load

                route['total_cbm_load'] =  total_cbm_load
                
                X  = route_value['plan_output'][-1]["leaving_time"]
                Y  = route_value['plan_output'][0]["arrival_time"]


                total_duration = convert_to_date_obj(X)[1]   -  convert_to_date_obj(Y)[1]  

                route['total_duration'] = total_duration

                route['total_distance'] = route_value['plan_output'][-1]['distance']

                route['elements'] = route_value['plan_output']

                solution['routes'].append(route)



        solution['unscheduled_requests'] = unscheduled_requests

        solutions.append(solution)

        output['solutions'] = solutions

        return output
    output = convert_to_output(dict_veh)
    
    return output



In [50]:
# customer_list_by_depot = VDC_new[1]['customer_list_by_depot']
# vehicle_list_by_depot = VDC_new[1]['vehicle_list_by_depot']
# depot_list_new = VDC_new[1]['depot_list_new']
# vehicle_list_after_split, customer_list_after_split,depot_list_after_split,input_after_split ,only_customer_list_after_split\
# = function_split_request(customer_list_by_depot,vehicle_list_by_depot,depot_list_new)

In [51]:
def main_latest(VDC_new):
    result_list = []
    for index_vdc in range(len(VDC_new)):

        customer_list_by_depot = VDC_new[index_vdc]['customer_list_by_depot']
        vehicle_list_by_depot = VDC_new[index_vdc]['vehicle_list_by_depot']
        depot_list_new = VDC_new[index_vdc]['depot_list_new']

        new_request_each_depot, input_after_split, start_LocationCode,end_LocationCode, vehicle_list_after_split,\
        depot_list_after_split, only_customer_list_after_split, customer_list_after_split, dict_to_list_id_split, arr_key_veh_split, \
        arr_key_split, dict_split_request_each_depot, merge_request_depot, total_service_time, DM_each_depot, weight_each_request, \
        dic_vehicle_after_split, fixed_time,matrix_service_time = function_split_request(customer_list_by_depot,vehicle_list_by_depot,depot_list_new)
        print("time___________________________________")
        print("total service time", total_service_time)
        print("fixed time", fixed_time)
        output = data_preprocessing(
            input_after_split, new_request_each_depot, start_LocationCode,end_LocationCode,
            vehicle_list_after_split, depot_list_after_split, only_customer_list_after_split,
            customer_list_after_split, dict_to_list_id_split, number_of_vehicle,
            arr_key_veh_split, arr_key_split, dict_split_request_each_depot, merge_request_depot,
            total_service_time, DM_each_depot, weight_each_request, dic_vehicle_after_split,
            fixed_time,vehicle_list_by_depot,matrix_service_time
        )
        result_list.append(output)
    return result_list

list_out = main_latest(VDC_new)







max_cbm <class 'numpy.int64'>
max_w <class 'numpy.int64'>
{('1003348', '4100', 0): {'items': [{'itemCode': '6610', 'quantity': 500, 'weight': 10000, 'cbm': 3538, 'quantityPerPallet': 0, 'size': {'width': 24.7, 'length': 37.3, 'height': 19.2}, 'iType': {'typeOfItemByVehicle': 'DEFAULT', 'typeOfItemByPaymentMethods': '', 'typeOfItemByPallet': 'Hang Xa'}, 'orderCode': 'testpackmore11'}, {'itemCode': '6610', 'quantity': 500, 'weight': 10000, 'cbm': 3538, 'quantityPerPallet': 0, 'size': {'width': 24.7, 'length': 37.3, 'height': 19.2}, 'iType': {'typeOfItemByVehicle': 'DEFAULT', 'typeOfItemByPaymentMethods': '', 'typeOfItemByPallet': 'Hang Xa'}, 'orderCode': 'testpackmore11'}, {'itemCode': '6610', 'quantity': 500, 'weight': 10000, 'cbm': 3538, 'quantityPerPallet': 0, 'size': {'width': 24.7, 'length': 37.3, 'height': 19.2}, 'iType': {'typeOfItemByVehicle': 'DEFAULT', 'typeOfItemByPaymentMethods': '', 'typeOfItemByPallet': 'Hang Xa'}, 'orderCode': 'testpackmore11'}, {'itemCode': '6610', 'quant

#test

In [52]:
A = list_out[0]['solutions'][0]['routes'][0]['elements']
B= [ [x['location_code'] , x['arrival_time'], x['leaving_time']]  for x in A]
B

[['8000000', '2023-04-02 08:10:41', '2023-04-02 08:10:41'],
 ['4100', '2023-04-02 08:37:47', '2023-04-02 18:52:47'],
 ['1003650', '2023-04-02 18:46:49', '2023-04-02 19:00:01'],
 ['1003577', '2023-04-02 20:17:13', '2023-04-02 20:36:25'],
 ['1003348', '2023-04-02 22:54:25', '2023-04-02 23:01:37'],
 ['8000000', '2023-04-03 00:00:00', '2023-04-03 00:00:00']]

In [53]:
def xulyoutput(list_out):
    output_finally = {'solutions': [  ] }
    dic_sol = { 'routes': [], 'additional_locations': [], 'unscheduled_requests': [] }
    for output in list_out:

        route = output['solutions'][0]['routes']

        unscheduled_request = output['solutions'][0]['unscheduled_requests']

        dic_sol['routes'].extend(route)

        dic_sol['unscheduled_requests'].extend(unscheduled_request)

    output_finally['solutions'].append(dic_sol)

    return output_finally

In [54]:
output_finally = xulyoutput(list_out)
output_finally

{'solutions': [{'routes': [{'vehicle_code': '95H-00399_000',
     'romooc_code': '61R-04839',
     'vehicle_weight': 40000000,
     'vehicle_cbm': 68300800,
     'total_weight_load': 30000000,
     'total_cbm_load': 5306500,
     'total_duration': 56959,
     'total_distance': 187520,
     'elements': [{'location_code': '8000000',
       'cd_code': '8000000',
       'distance': 0,
       'weight_load': 0,
       'cbm_load': 0,
       'arrival_time': '2023-04-02 08:10:41',
       'leaving_time': '2023-04-02 08:10:41',
       'location_type': 'STATION',
       'items': []},
      {'location_code': '4100',
       'cd_code': '4100',
       'distance': 0,
       'weight_load': 30000000,
       'cbm_load': 5306500,
       'arrival_time': '2023-04-02 08:37:47',
       'leaving_time': '2023-04-02 18:52:47',
       'location_type': 'DEPOT',
       'items': [{'item_code': '6612',
         'quantity': 1000,
         'weight': 10000000,
         'cbm': 1769000,
         'size': {'width': 24.7, 'le